In [1]:
%%writefile going_modular/data_setup.py

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

NUM_WORKER = os.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir: str,
    transforms: transforms.Compose,
    batch_size: int,
):
    train_data = datasets.ImageFolder(
        root = train_dir,
        transform = transforms,
        ) 
    test_data = datasets.ImageFolder(
        root = test_dir,
        transform = transforms
    )
    
    
    class_names = train_data.classes
    
    train_dataloader = DataLoader(
        dataset= train_data,
        batch_size= batch_size,
        shuffle= True,
        num_workers= NUM_WORKER,
        pin_memory= True
    )
    
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=NUM_WORKER,
        pin_memory=True
    )
    
    return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [ ]:
%%writefile going_modular/model_builder.py

import torch
import torch.nn as nn

class TinyVGG(nn.Module):
    def __init__(self, in_channels:int, out_shape:int, hidden_units:int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(num_features=hidden_units),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride = 1,
                      padding=1),
            nn.BatchNorm2d(num_features=hidden_units),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride = 2))
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*16*16,
                      out_features=out_shape)
        )
        
    def forward(self, X):
        X = self.conv_block_1(X)
        X = self.conv_block_2(X)
        X = self.classifier(X)
        return X

Writing going_modular/model_builder.py


In [43]:
%%writefile going_modular/engine.py

import test
import torch.utils.data.dataloader
from tqdm.auto import tqdm
import torch
import torch.nn as nn

def train_step(train_dataloader: torch.utils.data.DataLoader,
               model: nn.Module,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device ,
               
               ):
    
        train_loss = 0
        train_acc = 0
        model.train()
        for X,y in train_dataloader:
            X,y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            acc = (y_pred.argmax(dim=1) == y).sum().item() / len(y)
            train_loss += loss
            train_acc += acc
        train_acc /= len(train_dataloader)
        train_loss /= len(train_dataloader)
        return train_loss, train_acc
    
    
def test_step(model:nn.Module,
              loss_fn:nn.Module,
              test_dataloader:torch.utils.data.DataLoader,
              device:torch.device):
    model.eval()
    with torch.inference_mode():
        
        test_loss = 0 
        test_acc = 0
        for X,y in test_dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            acc = (y_pred.argmax(dim=1)== y).sum().item() / len(y)
            test_loss += loss
            test_acc += acc
        test_acc/= len(test_dataloader)
        test_loss /= len(test_dataloader)
    return test_loss, test_acc
        
        
def train(model: nn.Module,
          loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device:torch.device,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader):
    
    results = {
        "train_loss": [],
        "train_acc" : [],
        "test_loss" : [],
        "test_acc"  : []
    }
    for epoch in range(epochs):
        
        train_loss, train_acc =train_step(train_dataloader=train_dataloader,
                                            model= model,
                                            loss_fn=loss_fn,
                                            optimizer=optimizer,
                                            device=device)
        test_loss , test_acc = test_step(model= model,
                                         loss_fn=loss_fn,
                                         test_dataloader=test_dataloader,
                                         device=device)
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        print(f"EPOCH: {epoch+1}\n Train Loss : {train_loss} , Train Accuracy : {train_acc} | Test Loss : {test_loss} , Test Acc : {test_acc}")
    return results


Writing going_modular/engine.py


In [44]:
%%writefile going_modular/utils.py
import torch
from pathlib import Path

def save_model(model:torch.nn.Module,
               target_dir:str,
               model_name: str):
    target_dir_path = Path(target_dir)
    
    if target_dir_path.is_dir():
        print(f"{target_dir_path} already exists!!")
    else:
        target_dir_path.mkdir(parents = True, exist_ok=True)
        
    assert model_name.endswith(".pth") or model_name.endswith(".pt")
    model_save_path = target_dir_path / model_name
    torch.save(obj = model.state_dict(),
               f = model_save_path)        

Writing going_modular/utils.py
